In [1]:
import pandas
import pdb
import numpy
import logging



from pandas import DataFrame as df
from utils.pyconfig import PyConfig
from data.steersim import get_base_path, steersimProcess
from data.steersim_evac import SteersimEvacProcess


In [4]:
logging.basicConfig(level=logging.INFO)

root_dir = r'/home/kaidong/RubymineProjects/ActiveLoop/storage/scene-evac-1-base-data/train-20fps'
seq_name = 'Kaidong-Ubuntu-20230712-120319-29641'
get_base_path(f"{root_dir}/{seq_name}.bin")
cfg = PyConfig('steersim', 14, 13, 6, 4)
cfg.set("agent_num", 30)

data = SteersimEvacProcess(root_dir, seq_name, cfg, None)
data(30)['frame']


INFO:root:Group 65 has 30 agents, they are: 
[5, 6, 9, 10, 13, 15, 19, 24, 25, 26, 30, 35, 36, 37, 38, 41, 44, 46, 47, 51, 56, 57, 58, 59, 61, 62, 64, 65, 68, 70]
INFO:root:Group 1 has 30 agents, they are: 
[0, 1, 2, 5, 6, 10, 15, 18, 19, 20, 21, 22, 30, 31, 32, 34, 35, 36, 37, 38, 42, 44, 46, 47, 48, 61, 64, 65, 68, 70]
INFO:root:Cannot find enough agents to form a group, try incrementally
INFO:root:Cannot find enough agents to form a group, try incrementally
INFO:root:Cannot find enough agents to form a group, try incrementally
INFO:root:Group 39 has 30 agents, they are: 
[9, 13, 24, 25, 26, 37, 38, 39, 40, 41, 51, 52, 53, 54, 55, 56, 57, 58, 59, 61, 62, 64, 65, 66, 67, 68, 69, 70, 44, 15]
INFO:root:Cannot find enough agents to form a group, try incrementally
INFO:root:Cannot find enough agents to form a group, try incrementally
INFO:root:Cannot find enough agents to form a group, try incrementally
INFO:root:Cannot find enough agents to form a group, try incrementally
INFO:root:Canno

601

In [3]:
raise

RuntimeError: No active exception to reraise

In [ ]:
data.sub_dataset[0].get_max_total_frame()

In [ ]:
data.get_agent_num()

In [ ]:
d = pandas.DataFrame(data.gt[:, [0, 1, 15]], columns='nframe aid z'.split())
d['nframe'] = d.nframe.astype(int)
d['aid'] = d.aid.astype(int)
d['origin_index'] = d.index

clamp_max = d[d.z < 1].groupby('aid').nframe.max()
merged = pandas.merge(d, clamp_max, on="aid", suffixes=["", "_clamp"])
filtered = merged[merged.nframe > merged.nframe_clamp]

gt = numpy.take(data.gt, filtered.origin_index, axis=0)

In [ ]:
d = pandas.DataFrame(data.gt[:, [0, 1, 15]], columns='nframe aid z'.split())
d['nframe'] = d.nframe.astype(int)
d['aid'] = d.aid.astype(int)
d['origin_index'] = d.index

clamp_max = d[d.z < 1].groupby('aid').nframe.max()
merged = pandas.merge(d, clamp_max, on="aid", suffixes=["", "_clamp"])
filtered = merged[merged.nframe > merged.nframe_clamp]

gt = numpy.take(data.gt, filtered.origin_index, axis=0)

In [ ]:
numpy.all(gt[:, 15] > 1

In [ ]:
d.iloc[460]

In [ ]:
filtered.index.values

In [ ]:
numpy.all(gt[:, 15] > 1

In [ ]:
d.iloc[460]

In [ ]:
filtered.index.values

In [ ]:
"""
Given a set of trajectory, find overall n-nearest agents for given agent
Usage:
  # get 6 nearest agents for agent id 51
  AgentGrouping(data.gt).get_close_agents(51, 6)  
"""
class AgentGrouping:
    def __init__(self, gt):
        self._prepare(gt)
        
    @staticmethod
    def _compute_ADE(d, a1, a2):
        d1 = d[d.aid == a1]
        d2 = d[d.aid == a2]
        # get frames for both agents appears, align trajectory
        # and put trajectory data in x1, z1 / x2, z2
        dm = d1.merge(d2, on="nframe", suffixes=["1", "2"])
        if dm.empty:
            return numpy.Infinity
        t1 = dm[['x1', 'z1']].values  # extract values
        t2 = dm[['x2', 'z2']].values
        return numpy.mean(numpy.linalg.norm(t1-t2, axis=1))
    
    def _prepare(self, gt):
        # initialize dataframe used for pandas join
        colidx = [0, 1, 13, 15]
        cols = 'nframe aid x z'.split()
        self.d = pandas.DataFrame(gt[:, colidx], columns=cols)
        self.d['nframe'] = d.nframe.astype(int)
        self.d['aid'] = d.aid.astype(int)
        
        # discard initialize area and select main area only 
        self.d = self.d[self.d.z > 1]
        
        # cache ADE matrix so it can be reused
        nagents = self.d.aid.max() + 1
        self.ADEm = ADEm = numpy.zeros([nagents, nagents])
        # ADE matrix should be semmetric and main diagnonal should be 0s
        meshX, meshY = numpy.tril_indices_from(ADEm, k=-1)
        for x, y in zip(meshX, meshY):
            ADEm[y, x] = ADEm[x, y] = self._compute_ADE(self.d, x, y)
        
            
    def get_close_agents(self, target_idx, num_agent):
        return numpy.argpartition(self.ADEm[target_idx], num_agent)[:num_agent] 


In [ ]:
data.gt[:, [0, 1, 13, 15]]

In [ ]:
a = AgentGrouping(data.gt)

In [ ]:
numpy.take_along_axis(a.ADEm[51], numpy.argpartition(a.ADEm[51], 71), axis=0)

In [ ]:
a.ADEm[51, 0]

In [ ]:
dg1 = di[di.aid.isin([ 1,  3,  2,  0, 16, 30])]
dg1.plot.scatter('x','z',color=dg1.aid, cmap='rainbow')

In [ ]:
data.gt[:, [0, 1, 13, 15]]

In [ ]:
a = AgentGrouping(data.gt)

In [ ]:
numpy.take_along_axis(a.ADEm[51], numpy.argpartition(a.ADEm[51], 71), axis=0)

In [ ]:
a.ADEm[51, 0]

In [ ]:
dg1 = di[di.aid.isin([ 1,  3,  2,  0, 16, 30])]
dg1.plot.scatter('x','z',color=dg1.aid, cmap='rainbow')